In [1]:
import random

def toss(p):
  return random.randint(0, 100) <= p

class SwapOperator:
  def __init__(self, i, j):
    self.i = i
    self.j = j
  
  def get_max(self):
    return max(self.i, self.j)

  def __str__(self):
    pair = (self.i, self.j)
    return str(pair)

  def __repr__(self):
    pair = (self.i, self.j)
    return str(pair)

class SwapSequence:
  def __init__(self, seq):
    self.seq = seq # List of SwapOperator objects

  def iterate(self):
    for so in self.seq:
      yield so
  
  def add_swap_operator(self, so):
    self.seq.append(so)

  def get_highest_index(self):
    hi = 0
    for so in self.seq:
      hi = max(hi, so.get_max())
    return hi

  def copy(self):
    ret = []
    for so in self.seq:
      ret.append(SwapOperator(so.i, so.j))
    return SwapSequence(ret)

  def __add__(self, b): # Returns *Basic SwapSequence* that is equivalent to list(Sequence a) + list(Sequence b)
    c_seq = self.seq + b.seq
    c = SwapSequence(c_seq)
    n = c.get_highest_index() + 1

    initial_solution = Solution.gen_random_Solution(n)
    resultant_solution = initial_solution.add(c)

    return resultant_solution.sub(initial_solution)

  def __str__(self):
    return str(self.seq)

class Solution:
  def __init__(self, perm):
    self.perm = perm # A basic permutation

  def __add_SO(self, so, p):
    ret = self.copy()
    if toss(p):
      ret.perm[so.i], ret.perm[so.j] = ret.perm[so.j], ret.perm[so.i]
    return ret

  def __add_SS(self, ss, p):
    ret = self.copy()
    for so in ss.iterate():
      ret = ret.add(so, p)
    return ret

  def add(self, obj, p = 100):
    if isinstance(obj, SwapOperator):
      return self.__add_SO(obj, p)
    elif isinstance(obj, SwapSequence):
      return self.__add_SS(obj, p)
    else: return None
  
  def sub(self, b, p = 100):
    b = b.copy()
    a = self.copy()
    n = len(a.perm)
    ss = []
    for i in range(n):
      for j in range(n):
        if a.perm[i] == b.perm[j]:
          if i != j:
            so = SwapOperator(i, j)
            ss.append(so)
            b = b.add(so, p)
          else:
            break
    return SwapSequence(ss)

  def get(self, ind):
    return self.perm[ind]

  def copy(self):
    return Solution(self.perm[:])

  def __str__(self):
    return str(self.perm)

  def __eq__(self, b):
    for i in range(len(self.perm)):
      if self.get(i) != b.get(i):
        return False
    return True

  @classmethod
  def gen_random_Solution(cls, n):
    perm = [i for i in range(n)]
    random.shuffle(perm)
    return Solution(perm)

class Particle:
  def __init__(self, position, velocity, tsp):
    self.position = position
    self.p_best = self.position.copy()
    self.velocity = velocity
    self.fitness = tsp.apply_solution(position)
    self.p_best_fitness = self.fitness

  def get_fitness(self):
    return self.fitness
  
  def get_position(self):
    return self.position

  def get_p_best(self):
    return self.p_best

  def get_p_best_fitness(self):
    return self.p_best_fitness
  
  def update_velocity(self, g_best, alpha, beta):
    ini = self.velocity.copy()
    term1 = self.p_best.sub(self.position, alpha)
    term2 = g_best.sub(self.position, beta)
    self.velocity += term1 + term2

  def update_position(self, tsp):
    initial = self.position
    new = self.position.add(self.velocity)

    n_fitness = tsp.apply_solution(new)

    if n_fitness < self.get_p_best_fitness():
      self.p_best = new.copy()
      self.p_best_fitness = n_fitness

    self.fitness = n_fitness
    self.position = new

  def __str__(self):
    ret = f"Particle: {self.position}, fitness: {self.get_fitness()}, best_fitness: {self.get_p_best_fitness()}"
    return ret
  
  def __repr__(self):
    ret = f"Particle: {self.position}, fitness: {self.get_fitness()}, best_fitness: {self.get_p_best_fitness()}"
    return ret

  @classmethod
  def gen_random_Particle(self, n, tsp):
    position = Solution.gen_random_Solution(n)
    velocity = Solution.gen_random_Solution(n).sub(Solution.gen_random_Solution(n))
    return Particle(position, velocity, tsp)

class TSP:
  def __init__(self, g):
    self.g = g
    self.n = len(g)

  def apply_solution(self, s):
    cost = 0
    for i in range(self.n):
      cost += self.g[s.get(i)][s.get((i + 1) % self.n)]
    return cost
  
  @classmethod
  def gen_random_TSP(cls, n, soln):
    ret = []
    for i in range(n):
      row = []
      for j in range(n):
        row.append(random.randint(2, 10))
      ret.append(row)
      ret[i][i] = 0
    soln = soln.perm
    for i in range(n):
      ret[soln[i]][soln[(i+1)%n]] = 1
    return TSP(ret)

def get_g_best(particles, g_best):
  g_best, g_best_fitness = g_best
  for particle in particles:
    if particle.get_p_best_fitness() < g_best_fitness:
      g_best = particle.get_p_best().copy()
      g_best_fitness = particle.get_p_best_fitness()
  return g_best, g_best_fitness

ALPHA = 70
BETA = 80
POPULATION_SIZE = 20
NUM_NODES = 10
NUM_ITERATIONS = 1000

actual_solution = [i for i in range(NUM_NODES)]
random.shuffle(actual_solution)
actual_solution = Solution(actual_solution)

tsp = TSP.gen_random_TSP(NUM_NODES, actual_solution)
particles = [Particle.gen_random_Particle(NUM_NODES, tsp) for _ in range(POPULATION_SIZE)]
g_best_fitness = particles[0].get_fitness()
g_best, g_best_fitness = get_g_best(particles, (particles[0], g_best_fitness))

print("Initial Particles: ")
for particle in particles:
  print(particle)
print(f"Best Solution: {g_best}, Fitness: {g_best_fitness}")

for i in range(NUM_ITERATIONS):
  for particle in particles:
    particle.update_velocity(g_best, ALPHA, BETA)
    particle.update_position(tsp)
    g_best, g_best_fitness = get_g_best(particles, (g_best, g_best_fitness))
  print(f"Iteration {i}, Best Solution: {g_best}, Fitness: {g_best_fitness}")

print(actual_solution)
print(g_best)

Initial Particles: 
Particle: [1, 4, 7, 9, 2, 8, 6, 3, 0, 5], fitness: 46, best_fitness: 46
Particle: [8, 4, 2, 5, 1, 9, 3, 0, 6, 7], fitness: 48, best_fitness: 48
Particle: [2, 8, 9, 6, 7, 3, 0, 1, 5, 4], fitness: 43, best_fitness: 43
Particle: [7, 4, 6, 8, 5, 2, 3, 9, 0, 1], fitness: 55, best_fitness: 55
Particle: [5, 1, 9, 7, 2, 4, 0, 6, 3, 8], fitness: 57, best_fitness: 57
Particle: [5, 6, 2, 3, 4, 9, 7, 8, 0, 1], fitness: 58, best_fitness: 58
Particle: [4, 6, 5, 3, 8, 1, 2, 9, 7, 0], fitness: 49, best_fitness: 49
Particle: [4, 5, 7, 0, 8, 1, 3, 6, 9, 2], fitness: 51, best_fitness: 51
Particle: [7, 8, 3, 1, 9, 5, 6, 4, 2, 0], fitness: 43, best_fitness: 43
Particle: [6, 1, 0, 2, 9, 3, 4, 8, 5, 7], fitness: 52, best_fitness: 52
Particle: [8, 6, 9, 3, 7, 4, 1, 5, 0, 2], fitness: 56, best_fitness: 56
Particle: [0, 3, 1, 7, 9, 6, 8, 4, 5, 2], fitness: 46, best_fitness: 46
Particle: [2, 0, 3, 7, 4, 6, 8, 1, 5, 9], fitness: 64, best_fitness: 64
Particle: [5, 8, 0, 4, 6, 1, 9, 3, 2, 7], fi

Iteration 149, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 150, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 151, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 152, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 153, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 154, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 155, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 156, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 157, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 158, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 159, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 160, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 161, Best Solution: [9, 8, 7, 3, 0, 6, 4, 5, 1, 2], Fitness: 23
Iteration 162, Best Solution: [9, 8, 7

Iteration 340, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 341, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 342, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 343, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 344, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 345, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 346, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 347, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 348, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 349, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 350, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 351, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 352, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 353, Best Solution: [8, 6, 4

Iteration 489, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 490, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 491, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 492, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 493, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 494, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 495, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 496, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 497, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 498, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 499, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 500, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 501, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 502, Best Solution: [8, 6, 4

Iteration 760, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 761, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 762, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 763, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 764, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 765, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 766, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 767, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 768, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 769, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 770, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 771, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 772, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 773, Best Solution: [8, 6, 4

Iteration 907, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 908, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 909, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 910, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 911, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 912, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 913, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 914, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 915, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 916, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 917, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 918, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 919, Best Solution: [8, 6, 4, 2, 9, 1, 7, 3, 0, 5], Fitness: 18
Iteration 920, Best Solution: [8, 6, 4